In [ ]:
import tweepy
import spacy
import pandas as pd
import matplotlib.pyplot as plt
import time

from config import consumer_key, consumer_secret, access_token, access_token_secret

In [ ]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

# Load model
nlp = spacy.load('en')

In [ ]:
def post_analysis(username):

    # Create dictionary to hold text and label entities
    tweet_dict = {"text": [], "label": []}

    # Get tweets from a users timeline
    user_tweets = api.user_timeline(username)

    # Loop throught tweets
    for tweet in user_tweets:    
        doc = nlp(tweet["text"])
        if doc.ents:
            for ent in doc.ents:
                tweet_dict["text"].append(ent.text)
                tweet_dict["label"].append(ent.label_)

    # Convert dictionary to DataFrame
    tweet_df = pd.DataFrame(tweet_dict)
    tweet_df.head()
    
    # Group by labels
    label_frequency = tweet_df.groupby(['label']).count()

    # Show bar chart
    label_frequency.plot.bar()    

    # Save bar chart to png
    plt.savefig('chart.png')
    
    # Get bar graph as a figure and tweet chart
    api.update_with_media('chart.png', f'Tweet labels for @{username}')

In [ ]:
# post_analysis('CNN')

In [ ]:
me = 'TylerUCSD'

In [ ]:
def find_completed_requests():
    tweets = api.user_timeline(rpp=1000)

    completed_requests = set()
    for tweet in tweets:
        if 'labels for' not in tweet['text']:
            continue        
        for user_mention in tweet['entities']['user_mentions']:
            if user_mention['screen_name'] != me:
                completed_requests.add(user_mention['screen_name'])
                
    return completed_requests

completed_requests = find_completed_requests()
print('Completed Requests:', completed_requests)

In [ ]:
def find_next_request():
    tweets = api.search(f'@{me} Analyze:')['statuses']
    
    requests = set()
    for tweet in tweets:
        for user_mention in tweet['entities']['user_mentions']:
            if user_mention['screen_name'] != me:
                requests.add(user_mention['screen_name'])
        
    new_requests = requests - find_completed_requests()
    
    try:
        return new_requests.pop()
    except:
        return None
    
next_request = find_next_request()
print('Next Request:', next_request)

In [ ]:
if next_request:
    post_analysis(next_request)

In [ ]:
while True:
    print("Updating Twitter")

    next_request = find_next_request()
    print('Next Request:', next_request)
    
    if next_request:
        post_analysis(next_request)
    
    time.sleep(20)